In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = '3'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy as d_copy
import random

print(torch.__version__, torchvision.__version__)
print(torch.cuda.get_device_name(0))

1.8.0 0.2.2
GeForce RTX 3080


In [28]:
!ls /media/0/hwbae0326/F4F/models
model_path = "/media/0/hwbae0326/F4F/models/" 


 0~128_pkt_err_f4f_loss_0.000683_epoch_1.pt
 0~128_pkt_err_f4f_loss_0.002703_epoch_2.pt
 0~128_pkt_err_f4f_loss_0.006040_epoch_3.pt
 0~128_pkt_err_f4f_loss_1.835547_epoch_1.pt
 0~128_pkt_err_f4f_loss_1.940522_epoch_2.pt
 0~128_pkt_err_f4f_loss_2.034444_epoch_3.pt
 0~128_pkt_err_f4f_loss_2.099603_epoch_4.pt
 0~128_pkt_err_f4f_loss_2.137438_epoch_5.pt
 0~128_pkt_err_f4f_loss_2.159736_epoch_6.pt
 0~128_pkt_err_f4f_loss_2.189610_epoch_7.pt
 0~128_pkt_err_f4f_loss_2.206504_epoch_8.pt
 0~128_pkt_err_f4f_loss_2.233223_epoch_9.pt
 0~128_pkt_err_f4f_loss_2.240610_epoch_10.pt
 0~128_pkt_err_f4f_loss_2.248619_epoch_11.pt
 0~128_pkt_err_f4f_loss_2.248688_epoch_20.pt
 0~128_pkt_err_f4f_loss_2.249367_epoch_16.pt
 0~128_pkt_err_f4f_loss_2.249795_epoch_19.pt
 0~128_pkt_err_f4f_loss_2.251496_epoch_12.pt
 0~128_pkt_err_f4f_loss_2.252112_epoch_18.pt
 0~128_pkt_err_f4f_loss_2.253447_epoch_21.pt
 0~128_pkt_err_f4f_loss_2.256312_epoch_17.pt
 0~128_pkt_err_f4f_loss_2.256330_epoch_24.pt
 0~128_pkt_err_f4f_los

In [45]:
model_list = os.listdir(model_path)
model_list = list(filter(lambda x : "0~128" in x,model_list))
model_list = list(filter(lambda x : "new" not in x,model_list))
model_list

['0~128_pkt_err_f4f_loss_2.256312_epoch_17.pt',
 '0~128_pkt_err_f4f_loss_2.206504_epoch_8.pt',
 '0~128_pkt_err_f4f_loss_2.257969_epoch_15.pt',
 '0~128_pkt_err_f4f_loss_2.256330_epoch_24.pt',
 '0~128_pkt_err_f4f_loss_2.249367_epoch_16.pt',
 'new_  0~128_pkt_err_f4f_loss_1.643100_epoch_ 1.pt',
 '0~128_pkt_err_f4f_loss_2.264266_epoch_26.pt',
 '0~128_pkt_err_f4f_loss_2.275270_epoch_30.pt',
 'new_  0~128_pkt_err_f4f_loss_2.245951_epoch_16.pt',
 'new_  0~128_pkt_err_f4f_loss_2.256897_epoch_ 9.pt',
 'new_  0~128_pkt_err_f4f_loss_2.180818_epoch_ 4.pt',
 'new_  0~128_pkt_err_f4f_loss_2.242200_epoch_12.pt',
 '0~128_pkt_err_f4f_loss_2.258156_epoch_23.pt',
 '0~128_pkt_err_f4f_loss_2.252112_epoch_18.pt',
 'new_  0~128_pkt_err_f4f_loss_1.972052_epoch_ 2.pt',
 '0~128_pkt_err_f4f_loss_2.265946_epoch_27.pt',
 '0~128_pkt_err_f4f_loss_1.835547_epoch_1.pt',
 '0~128_pkt_err_f4f_loss_2.233223_epoch_9.pt',
 '0~128_pkt_err_f4f_loss_2.240610_epoch_10.pt',
 'new_  0~128_pkt_err_f4f_loss_2.241876_epoch_15.pt',
 

In [69]:
model_epoch6 = list(filter(lambda x : "epoch_6" in x, model_list))
model_epoch20 = list(filter(lambda x : "epoch_20" in x, model_list)) # epoch_10 -> torch.equal : True
model_epoch6 = model_epoch6[0]
model_epoch20 = model_epoch20[0]

In [70]:
epoch6_weight = torch.load(model_path+model_epoch6)
epoch6_weight

Parameter containing:
tensor([[-0.0123, -0.0123,  0.0090,  ...,  0.0098, -0.0077,  0.0085],
        [-0.0086, -0.0134, -0.0035,  ...,  0.0132, -0.0038,  0.0007],
        [-0.0089,  0.0094,  0.0048,  ..., -0.0097,  0.0018, -0.0042],
        ...,
        [ 0.0097,  0.0108,  0.0011,  ...,  0.0023,  0.0050,  0.0099],
        [-0.0137,  0.0027, -0.0062,  ..., -0.0119, -0.0055,  0.0096],
        [-0.0110, -0.0056,  0.0100,  ..., -0.0071,  0.0020,  0.0037]],
       device='cuda:0', requires_grad=True)

In [71]:
epoch20_weight = torch.load(model_path+model_epoch20)
epoch20_weight

Parameter containing:
tensor([[ 0.0041, -0.0105, -0.0133,  ...,  0.0140,  0.0128, -0.0041],
        [-0.0089, -0.0104, -0.0056,  ...,  0.0115,  0.0097,  0.0138],
        [-0.0069,  0.0094,  0.0136,  ..., -0.0102, -0.0085,  0.0032],
        ...,
        [ 0.0138,  0.0035, -0.0064,  ...,  0.0089, -0.0029,  0.0009],
        [ 0.0050,  0.0069,  0.0109,  ..., -0.0017,  0.0136,  0.0037],
        [-0.0023,  0.0098, -0.0134,  ..., -0.0140, -0.0091,  0.0095]],
       device='cuda:0', requires_grad=True)

In [73]:
torch.equal(epoch6_weight,epoch20_weight)

False

In [68]:
new_model_list = list(filter(lambda x : "new" in x,model_list))
new_model_list

['new_  0~128_pkt_err_f4f_loss_1.643100_epoch_ 1.pt',
 'new_  0~128_pkt_err_f4f_loss_2.245951_epoch_16.pt',
 'new_  0~128_pkt_err_f4f_loss_2.256897_epoch_ 9.pt',
 'new_  0~128_pkt_err_f4f_loss_2.180818_epoch_ 4.pt',
 'new_  0~128_pkt_err_f4f_loss_2.242200_epoch_12.pt',
 'new_  0~128_pkt_err_f4f_loss_1.972052_epoch_ 2.pt',
 'new_  0~128_pkt_err_f4f_loss_2.241876_epoch_15.pt',
 'new_  0~128_pkt_err_f4f_loss_2.251746_epoch_19.pt',
 'new_  0~128_pkt_err_f4f_loss_0.330272_epoch_ 1.pt',
 'new_  0~128_pkt_err_f4f_loss_2.242217_epoch_14.pt',
 'new_  0~128_pkt_err_f4f_loss_2.221348_epoch_ 5.pt',
 'new_  0~128_pkt_err_f4f_loss_2.250254_epoch_18.pt',
 'new_  0~128_pkt_err_f4f_loss_2.254653_epoch_ 7.pt',
 'new_  0~128_pkt_err_f4f_loss_2.108496_epoch_ 3.pt',
 'new_  0~128_pkt_err_f4f_loss_2.251714_epoch_17.pt',
 'new_  0~128_pkt_err_f4f_loss_2.250370_epoch_11.pt',
 'new_  0~128_pkt_err_f4f_loss_2.258917_epoch_ 8.pt',
 'new_  0~128_pkt_err_f4f_loss_2.252817_epoch_20.pt',
 'new_  0~128_pkt_err_f4f_lo

In [66]:
new_model_epoch6 = list(filter(lambda x : "epoch_"))

TypeError: filter expected 2 arguments, got 1